In [ ]:
import os
import numpy as np
import pandas as pd
import einops
import torch 

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from tqdm import tqdm

import pyarrow as pa
import pyarrow.parquet as pq

In [ ]:
import datetime

# Compute unix epoch to date table

def _prep_unix_epoch_to_date(max_year = 2025):
    month_abbr = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    max_day = (datetime.datetime(max_year, 1, 1, 0, 0) - datetime.datetime(1970, 1, 1, 0, 0)).days 
    unix_epoch = [i for i in range(max_day)]
    date_times = [datetime.datetime(1970, 1, 1, 0, 0) + datetime.timedelta(i) for i in range(max_day)]

    def as_SowDate(date_time):
        day = f'{date_time.day}'
        if len(day) == 1:
            return(f'0{day}-{month_abbr[date_time.month - 1]}')
        else:
            return(f'{day}-{month_abbr[date_time.month - 1]}')

    tmp = pd.DataFrame({
        'Unix': unix_epoch,
        'Year':[e.year for e in date_times],
        'Month':[e.month for e in date_times],
        'Day':[e.day for e in date_times],
        'SowDate':[as_SowDate(date_time = e) for e in date_times]
    })
    _ = tmp.loc[:, ['Unix', 'Year']].groupby(['Year']).min().reset_index().rename(columns={'Unix':'MinUnix'})
    tmp = tmp.merge(_)

    tmp['DOY'] = tmp['Unix'] - tmp['MinUnix']
    tmp = tmp.drop(columns = ['MinUnix'])
    return tmp

In [ ]:
# how bad would it be if we stored a bunch of tiny parquet files? 
# saving many tiny parquet files will increase storage cost by 1.6x
# That's not great but not horrible either. We're talking about approximately 500 gb.
# (305213583096*1.6)/1000000000
# 488.34

# apsimx_sim_parquet_dir = '/home/Shared/cultivar_sim_exps'
# Result = pq.read_table(apsimx_sim_parquet_dir+'/'+'sim_1698440407_4739.parquet').to_pandas()

# -rw-rw-r-- 1 kickd newgroup 2939581425 Jun  1 01:08 cultivar_sim_exps/sim_1698440407_4739.parquet


# for i in tqdm(Result.FactorialUID.unique()):
#     table = Result.loc[(Result.FactorialUID == i), ].drop(columns = 'FactorialUID')
#     table = pa.Table.from_pandas(table)
#     pq.write_table(table, f'/home/Shared/testing_rm_after0620/{i}.parquet')

# 100%|██████████| 24025/24025 [33:58<00:00, 11.79it/s]

In [ ]:
# The goal is to have a (postgres?) SQL db that we can query. To not have a delay we're going to instead load a batch into memory BUT allow for redefining this batch by swapping out the dataloader.

# Workflow:
# Define the desired data 
# Use the main tables to figure out what parquet files we need to pull from.
# Pull all the data in and represent as tensors

In [ ]:
apsimx_sim_parquet_dir = '/home/Shared/cultivar_sim_exps'
os.listdir(apsimx_sim_parquet_dir)[0:3]

['sim_1698440407_4739.parquet',
 'sim_1697418008_10643.parquet',
 'sim_1697187607_79518.parquet']

In [ ]:
[e for e in os.listdir(apsimx_sim_parquet_dir) if e[0:4] != 'sim_']

['DefaultCultivarsAll.parquet', 'Genotypes.parquet', 'Ids.parquet']

In [ ]:
# metadata 
DefaultCultivarsAll = pq.read_table(apsimx_sim_parquet_dir+'/'+'DefaultCultivarsAll.parquet').to_pandas()
Genotypes           = pq.read_table(apsimx_sim_parquet_dir+'/'+'Genotypes.parquet').to_pandas()
Ids                 = pq.read_table(apsimx_sim_parquet_dir+'/'+'Ids.parquet').to_pandas()

In [ ]:
# I'm setting up a class to help find the files we need to read through to build the datset 
# This works by holding a copy of the Ids and Genotypes tables. 
# We'll operate on those, filtering them down until the tables only contain the enries we want to use.
# Next we'll use a that produces tuples of the (parquet file, filtering criteria)

class data_helper():
    def __init__(self, genotypes_path, ids_path, results_path, ssurgo_path, met_path):
        # used later to get the results. Append / if there isn't one.
        if results_path[-1] != '/': 
            results_path = results_path+'/'
        self.results_path = results_path

        Genotypes = pq.read_table(genotypes_path).to_pandas()
        # coerce None to NaN (default cultivars don't have all values specified)
        for e in [ee for ee in list(Genotypes) if ee not in ['File', 'Genotype']]:
            Genotypes[e] = Genotypes[e].astype(float)
        mask = (Genotypes.isna().sum(axis = 1) == 0)

        self.Genotypes = Genotypes.loc[mask, ].reset_index(drop = True)
        self.Ids = pq.read_table(ids_path).to_pandas()

        # Environmental data
        self.ssurgo_data = pq.read_table(ssurgo_path
                            ).to_pandas(
                            ).rename(columns = {'soil_i':'SoilIdx'})
        
        self.met_data    = pq.read_table(met_path
                            ).to_pandas(
                            ).rename(columns = {'latitude':  'Latitude',
                                                'longitude': 'Longitude'})

    
    def apply_mask(self, table, mask):
        "This method takes care of automatically filtering the non-masked table."
        if table not in ['Genotypes', 'Ids']:
            print('table should be Genotypes or Ids')   
        else:
            if table == 'Genotypes':
                # apply mask to filter the table
                self.Genotypes = self.Genotypes.loc[mask, ].reset_index(drop = True)
                # left join to update the other table
                self.Ids = self.Genotypes.loc[:, ['File', 'Genotype']
                                        ].drop_duplicates(
                                        ).merge(self.Ids, how = 'left'
                                        ).reset_index(drop = True)
            elif table == 'Ids':
                self.Ids = self.Ids.loc[mask, ].reset_index(drop = True)
                self.Genotypes = self.Ids.loc[:, ['File', 'Genotype']
                                    ].drop_duplicates(
                                    ).merge(self.Genotypes, how = 'left'
                                    ).reset_index(drop = True)
    def load_results(self, years = [], dry_run = True):
        self.years = years
        # Now we can ask for the files that we should get
        tmp = self.Ids.loc[:, ['File', 'Genotype', 'FactorialUID']]

        get_files = tmp.File.drop_duplicates().to_list()
        print(f'{len(get_files)} files to be read.')

        if dry_run == True:
            print('In dry_run, reading no files')

        if dry_run == False:
            res_list = []
            col_order = ''
            for file in tqdm(get_files):
                # print(f'{file}')
                res = pq.read_table(f'{x.results_path+file}.parquet').to_pandas()

                # columns should be in the same order, but we will force them to be here. 
                if col_order == '':
                    col_order = list(res)

                # filter order established with some informal testing.
                # runtime filter years, factorials: [10, 11.3, 10.5]
                # runtime filter factorials, years: [9.0, 8.4, 9.6]

                # filter factorials
                res = tmp.loc[(tmp.File == file), ['FactorialUID']].merge(res)

                # filter years
                if years != []:
                    yr = _prep_unix_epoch_to_date(max_year = 2030)
                    yr = yr.loc[(yr.Year.isin(years)), ['Unix']].rename(columns={'Unix':'Date'})
                    res = yr.merge(res)

                res_list.append(res)
            res_list = pd.concat(res_list).reset_index(drop=True)
            self.results = res_list

    def setup_encoders(self):
        # create string to num encoders so we can use tensors for all lookups
        Ids = self.Ids
        self.encoder_Genotype = {k:v for k,v in zip(Ids.Genotype.unique(), range(len(Ids.Genotype.unique())))}
        self.encoder_File     = {k:v for k,v in zip(Ids.File.unique(),     range(len(Ids.File.unique())))}
        self.encoder_SowDate  = {k:v for k,v in zip(Ids.SowDate.unique(),  range(len(Ids.SowDate.unique())))}

    def results_as_arrays(self, output_type = 'point'): #output_type = 'sequence'
        metadata = ['FactorialUID', 'Year']
        metrics  = ['Maize.AboveGround.Wt', 'Maize.LAI', 'yield_Kgha']
        ref = {
            'lookup_names': metadata,
            'data_names': metrics,
        }

        # Turn results into tensor
        tmp = self.results.merge(_prep_unix_epoch_to_date(max_year = 2025).loc[:, ['Unix', 'Year', 'DOY']].rename(columns={'Unix':'Date'}))

        # find the smallest above ground weight within each year, usethat to get the first date within eacy year and then filter for the values that are >= the within year start.
        _ = tmp.loc[:, metadata+['Maize.AboveGround.Wt']].groupby(metadata).agg('min').reset_index()
        _ = _.merge(tmp).loc[:, metadata+['Date']].rename(columns={'Date': 'YearStart'})
        tmp = tmp.merge(_, how='left')

        tmp = tmp.loc[(tmp.Date >= tmp.YearStart), ]

        tmp=tmp.drop(columns=['YearStart', 'Date']
            ).drop_duplicates(
            ).sort_values(metadata+['DOY']
            ).reset_index(drop = True)

        if output_type not in ['point', 'sequence']:
            print('output_type expected to be point or sequence')
        if output_type == 'point':
            ref['data_dims'] = ['obs', 'metrics']
            # if making point estimates:
            tmp = tmp.drop(columns='DOY').groupby(metadata).agg('max').reset_index()

            lookup = torch.tensor(tmp.loc[:, metadata].to_numpy())
            out = torch.tensor(tmp.drop(columns=metadata).to_numpy())
        
        if output_type == 'sequence':
            ref['data_dims'] = ['obs', 'days', 'metrics']
            day_lookup = _prep_unix_epoch_to_date(max_year = 2025).rename(columns={'Unix':'Date'})
            # day_lookup
            lookup = tmp.loc[:, metadata].drop_duplicates().reset_index(drop = True)
            
            # obs, channels, length
            out = torch.zeros((lookup.shape[0], 
                            365,
                            3
                            ))
            
            for i in tqdm(range(lookup.shape[0])):
                # break
                uid, yr = lookup.loc[i, ]

                # use Ids table to get planting date. 
                sow_date = self.Ids.loc[(self.Ids.FactorialUID == uid), 'SowDate']

                plant_DOY = day_lookup.loc[(
                    (day_lookup.Year == yr) & 
                    (day_lookup.SowDate == sow_date.values[0])), 'DOY'].values[0]


                mask = (
                    (tmp.FactorialUID == uid) & 
                    (tmp.Year == yr) &
                    (tmp.DOY >= plant_DOY) & 
                    (tmp.DOY <= 364) # because there are leap years and indexing starts at 0, clip to the minimum of these
                    )
                
                start_DOY= tmp.loc[mask, 'DOY'].min()
                stop_DOY = tmp.loc[mask, 'DOY'].max()

                out[i, start_DOY:(stop_DOY+1), :] = torch.tensor(tmp.loc[mask, metrics].to_numpy())
                # # propagate forward observations to all dates following the max
                out[i, stop_DOY:365, :] = out[i, stop_DOY, :]
            
        return(lookup, out, ref)  


    # Working with Environmental Data
    def filter_env(self):
        # filter to selected years
        if self.years != []:
            self.met_data = self.met_data.loc[(self.met_data.year.isin(self.years)), ].reset_index(drop = True)

        # shared soilIdx
        Ids_idxs = self.Ids.loc[:, ['Longitude', 'Latitude', 'SoilIdx']].drop_duplicates()
        ssurgo_s = self.ssurgo_data.loc[:, ['SoilIdx']].drop_duplicates()
        met_lonlat = self.met_data.loc[:, ['Longitude', 'Latitude']].drop_duplicates()

        print(f'Uniq. Lon/Lat/Soil: {Ids_idxs.shape[0]}')
        print('Condition on env availability:')
        Ids_idxs = Ids_idxs.merge(ssurgo_s, how = 'inner').merge(met_lonlat, how = 'inner')
        print(f'Uniq. Lon/Lat/Soil: {Ids_idxs.shape[0]}')

        # reduce the environmental datasets:
        self.met_data    = Ids_idxs.drop(columns=['SoilIdx']).drop_duplicates().merge(self.met_data, how = 'inner')
        self.ssurgo_data = Ids_idxs.drop(columns=['Longitude', 'Latitude']).drop_duplicates().merge(self.ssurgo_data, how = 'inner')

        # update Ids & results:
        self.Ids = Ids_idxs.merge(self.Ids)
        _ = self.Ids.loc[:, ['FactorialUID']].drop_duplicates()
        self.results = _.merge(self.results, how='inner')

        # refresh index on everything
        self.Ids         = self.Ids.reset_index(        drop = True)
        self.results     = self.results.reset_index(    drop = True)
        self.Genotypes   = self.Genotypes.reset_index(  drop = True)
        self.met_data    = self.met_data.reset_index(   drop = True)
        self.ssurgo_data = self.ssurgo_data.reset_index(drop = True)

    def met_as_arrays(self, ops_string = 's d m -> s d m'): # site day metric
        m = self.met_data.copy()
        m = m.rename(columns = {'year': 'Year'})
        metadata = [
            'Longitude', 
            'Latitude', 
            'Year'
            ]
        metrics = [
            'radn_MJ_div_m2_div_day',
            'maxt_oC',
            'mint_oC',
            'rain_mm',
            'rh_pr',
            'windspeed_m_div_s',
            'tav',
            'amp'
            ]


        # make sure that there are the expected number of values per group
        m = m.loc[(m.day < 366), ].sort_values(metadata).reset_index(drop = True)
        # there's at least one site that appears to have at least two records. I'll deal with this by (tav and amp appear to be slightly different)
        # see m.loc[((m.Longitude == -86.529600) & (m.Latitude == 34.729520) & (m.year == 2014) ), ].drop_duplicates().sort_values('day')

        m = m.groupby(metadata+['day']).mean().reset_index()

        # all groups have 365 obs?
        assert False not in (m.groupby(metadata).count().reset_index().loc[:, ['day']] == 365).values

        lookup = m.loc[:, metadata].drop_duplicates().to_numpy()

        m = m.loc[:, metrics
            ].to_numpy(
            ).reshape(
                -1,           # as many sites as are available
                365,          # days
                len(metrics)) # metrics


        m = einops.rearrange(m, ops_string)

        # let's also allow for an operations string so that we can _request_ data in a specific format instead of changing it after the fact.
        ops = ops_string.replace(' ', '').split('->')
        dim_order = [i
        for e in ops[1]
        for i in range(len(ops[0]))
        if e == ops[0][i]
        ]

        ref = {
            'lookup_names': metadata,
            'data_names': metrics,
            'data_dims': [['site', 'days', 'metrics'][i] for i in dim_order] # use the ops string to figure out the order that will be returned
        }

        return (lookup, m, ref)

    def ssurgo_as_arrays(self, ops_string = 's l m -> s l m'): # site lenght (depth) metric
        s = self.ssurgo_data.copy()

        metadata = [
            'SoilIdx',
            ]
        metrics = [
            'BD',
            'AirDry',
            'LL15',
            'DUL',
            'SAT',
            'KS',
            'Carbon',
            'SoilCNRatio',
            'FOM',
            'FOM.CN',
            'FBiom',
            'FInert',
            'NO3N',
            'NH4N',
            'PH',
            'ParticleSizeClay',
            'ParticleSizeSilt',
            'ParticleSizeSand',
            'Maize.KL',
            'Maize.LL',
            'Maize.XF',
            # 'Soybean.KL',
            # 'Soybean.LL',
            # 'Soybean.XF',
            # 'Wheat.KL',
            # 'Wheat.LL',
            # 'Wheat.XF'
            ]

        # check that the soil slices are equally sized
        assert len(s.Thickness.unique()) == 1
        print(f'Soil compartment thickness: {s.Thickness[0]}')

        # check that there are an equal number of slices
        _  = s.loc[:, ['SoilIdx', 'Thickness']].groupby(['SoilIdx']).count().reset_index()
        assert _.Thickness.min() == _.Thickness.max()

        # turn range into start of slice
        s['Depth'] = s['Depth'].str.split(pat = '-', expand = True)[0].astype(int)
        s = s.sort_values(['SoilIdx', 'Depth']).reset_index(drop = True)

        lookup = s.loc[:, metadata].drop_duplicates().to_numpy()

        s = s.loc[:, metrics].to_numpy()
        s = s.reshape(-1, lookup.shape[0], len(metrics))


        s = einops.rearrange(s, ops_string)

        # let's also allow for an operations string so that we can _request_ data in a specific format instead of changing it after the fact.
        ops = ops_string.replace(' ', '').split('->')
        dim_order = [i
        for e in ops[1]
        for i in range(len(ops[0]))
        if e == ops[0][i]
        ]


        ref = {
            'lookup_names': metadata,
            'data_names': metrics,
            'data_dims': [['site', 'depth', 'metrics'][i] for i in dim_order] # use the ops string to figure out the order that will be returned
        }
        return (lookup, s, ref)
    
    def genotype_as_arrays(self):
        g = self.Genotypes.copy()

        # Genotypes are easy to deal with because the table is cleaned up in filtering results. 
        metadata = [
            'File',
            'Genotype',
            ]
        metrics = [
            'Grain.MaximumGrainsPerCob.FixedValue',
            'Grain.MaximumPotentialGrainSize.FixedValue',
            'Phenology.FlagLeafToFlowering.Target.FixedValue',
            'Phenology.FloweringToGrainFilling.Target.FixedValue',
            'Phenology.GrainFilling.Target.FixedValue',
            'Phenology.Juvenile.Target.FixedValue',
            'Phenology.Maturing.Target.FixedValue',
            'Phenology.MaturityToHarvestRipe.Target.FixedValue',
            'Phenology.Photosensitive.Target.XYPairs.X__1',
            'Phenology.Photosensitive.Target.XYPairs.X__2',
            'Phenology.Photosensitive.Target.XYPairs.X__3',
            'Phenology.Photosensitive.Target.XYPairs.Y__1',
            'Phenology.Photosensitive.Target.XYPairs.Y__2',
            'Phenology.Photosensitive.Target.XYPairs.Y__3',
            'Rachis.DMDemands.Structural.DMDemandFunction.MaximumOrganWt.FixedValue',
        ]

        lookup = g.loc[:, metadata]
        lookup.File     = [self.encoder_File[e]     for e in lookup.File]
        lookup.Genotype = [self.encoder_Genotype[e] for e in lookup.Genotype]
        lookup = lookup.to_numpy()

        g = g.loc[:, metrics].to_numpy()

        ref = {
            'lookup_names': metadata,
            'data_names': metrics,
            'data_dims': ('genotype', 'metrics')
        }
        return(lookup, g, ref)

x = data_helper(
    genotypes_path = apsimx_sim_parquet_dir+'/'+'Genotypes.parquet',
    ids_path = apsimx_sim_parquet_dir+'/'+'Ids.parquet',
    results_path = apsimx_sim_parquet_dir,
    ssurgo_path = '/home/Shared/apsimx_env_data/apsimx_i_soil_ssurgo_profile.parquet',
    met_path    = '/home/Shared/apsimx_env_data/apsimx_i_soil_POWER_met.parquet'
    )

# restrict to simulated cultivars with the maximum MaximumGrainsPerCob
mask = (x.Genotypes['Grain.MaximumGrainsPerCob.FixedValue'] == x.Genotypes['Grain.MaximumGrainsPerCob.FixedValue'].max())

print(f'Before Mask: Genotypes: {x.Genotypes.shape} Ids: {x.Ids.shape}')
x.apply_mask(table='Genotypes', mask=mask)
print(f'After Mask: Genotypes: {x.Genotypes.shape} Ids: {x.Ids.shape}')

x.setup_encoders()
x.load_results(years = [1990, 2000, 2010, 2020], dry_run = False)
x.filter_env()

Before Mask: Genotypes: (3150, 17) Ids: (2837275, 7)
After Mask: Genotypes: (5, 17) Ids: (4247, 7)
5 files to be read.


100%|██████████| 5/5 [00:25<00:00,  5.05s/it]


Uniq. Lon/Lat/Soil: 126
Condition on env availability:
Uniq. Lon/Lat/Soil: 109


In [ ]:
result_lookup,   result,   result_ref =  x.results_as_arrays(output_type = 'point')

genotype_lookup, genotype, genotype_ref =  x.genotype_as_arrays()
met_lookup,      met,      met_ref =  x.met_as_arrays(   ops_string = 's d m -> s d m')
ssurgo_lookup,   ssurgo,   ssurgo_ref =  x.ssurgo_as_arrays(ops_string = 's l m -> s l m')

Soil compartment thickness: 200.0


In [ ]:
# x.Ids

# result_lookup   ['FactorialUID',                                                         'Year'],
# x.Ids           ['FactorialUID', 'File', 'Genotype', 'SoilIdx', 'Longitude', 'Latitude',          'SowDate', ]
# genotype_lookup [                'File', 'Genotype'],
# ssurgo_lookup   [                                    'SoilIdx']
# met_lookup      [                                               'Longitude', 'Latitude', 'year'],


day_lookup = _prep_unix_epoch_to_date(max_year = 2025).rename(columns={'Unix':'Date'})
day_lookup = day_lookup.loc[:, ['Year', 'SowDate', 'DOY']]

# encode text in tables
_ = list(set(day_lookup.SowDate))
encoder_SowDate_to_num = {_[i]:i for i in range(len(_))}
# encoder_num_to_SowDate = {encoder_SowDate_to_num[k]:k for k in encoder_SowDate_to_num}

In [ ]:
# Ids =  x.Ids.copy()

i = 0

# result_lookup[i]

Ids =  x.Ids.copy()

# create string to num encoders so we can use tensors for all lookups
encoder_Genotype = {k:v for k,v in zip(Ids.Genotype.unique(), range(len(Ids.Genotype.unique())))}
encoder_File = {k:v for k,v in zip(Ids.File.unique(), range(len(Ids.File.unique())))}
encoder_SowDate = {k:v for k,v in zip(Ids.SowDate.unique(), range(len(Ids.SowDate.unique())))}


# apply encoder
Ids.Genotype = [encoder_Genotype[e] for e in Ids.Genotype]
Ids.File     = [encoder_File[e]     for e in Ids.File]
Ids.SowDate  = [encoder_SowDate[e]  for e in Ids.SowDate]

ref = {'lookup_names': list(Ids),}


# (Ids.FactorialUID == result_lookup[i, 0])

# File

Ids

,Longitude,Latitude,SoilIdx,File,Genotype,SowDate,FactorialUID
0,-84.497556,42.689234,155,0,0,0,1
1,-84.497556,42.689234,155,0,0,1,2
2,-84.497556,42.689234,155,0,0,2,3
3,-84.497556,42.689234,155,0,0,3,4
4,-84.497556,42.689234,155,0,0,4,5
...,...,...,...,...,...,...,...
3622,-93.416595,40.746002,53,1,1,0,16674
3623,-93.416595,40.746002,53,1,1,4,16675
3624,-93.416595,40.746002,53,1,1,2,16676
3625,-93.416595,40.746002,53,1,1,5,16677


In [ ]:
x.Ids

In [ ]:
# recreate data_helper so that we can look at how the methods in data_helper work
x = data_helper(
    genotypes_path = apsimx_sim_parquet_dir+'/'+'Genotypes.parquet',
    ids_path = apsimx_sim_parquet_dir+'/'+'Ids.parquet',
    results_path = apsimx_sim_parquet_dir,
    ssurgo_path = '/home/Shared/apsimx_env_data/apsimx_i_soil_ssurgo_profile.parquet',
    met_path    = '/home/Shared/apsimx_env_data/apsimx_i_soil_POWER_met.parquet'
    )

# restrict to simulated cultivars with the maximum MaximumGrainsPerCob
mask = (x.Genotypes['Grain.MaximumGrainsPerCob.FixedValue'] == x.Genotypes['Grain.MaximumGrainsPerCob.FixedValue'].max())
x.apply_mask(table='Genotypes', mask=mask)

In [ ]:
m = x.met_data
metadata = [
    'Longitude', 
    'Latitude', 
    'year'
    ]
metrics = [
    'radn_MJ_div_m2_div_day',
    'maxt_oC',
    'mint_oC',
    'rain_mm',
    'rh_pr',
    'windspeed_m_div_s',
    'tav',
    'amp'
    ]

# make sure that there are the expected number of values per group
m = m.loc[(m.day < 366), ].sort_values(metadata).reset_index(drop = True)
m = m.groupby(metadata+['day']).mean().reset_index()

# all groups have 365 obs?
assert False not in (m.groupby(metadata).count().reset_index().loc[:, ['day']] == 365).values

In [ ]:
# Demonstrate reshaping. We want to "fold" this table so that there is a site (lon/lat/year) axis, a day axis, and a metrics axis.

# To demonstrate this we'll get the 
mn = m.loc[:, metadata+['day']].to_numpy()

mn = mn.reshape(-1,  # as many sites as are available
                365, # days
                4)   # metrics (in this case lon, lat, year, day)

import plotly.express as px
print('Here we\'ll show that this reshape is working as expected by plotting the "day" column across several sites')
px.imshow(
    np.concatenate([
        mn[i, 0:10, 3:]
        for i in range(10)
    ], axis = 1)
)

In [ ]:
m = x.met_data
metadata = [
    'Longitude', 
    'Latitude', 
    'year'
    ]
metrics = [
    'radn_MJ_div_m2_div_day',
    'maxt_oC',
    'mint_oC',
    'rain_mm',
    'rh_pr',
    'windspeed_m_div_s',
    'tav',
    'amp'
    ]


# make sure that there are the expected number of values per group
m = m.loc[(m.day < 366), ].sort_values(metadata).reset_index(drop = True)
m = m.groupby(metadata+['day']).mean().reset_index()

# all groups have 365 obs?
assert False not in (m.groupby(metadata).count().reset_index().loc[:, ['day']] == 365).values

In [ ]:

m = m.loc[:, metrics
    ].to_numpy(
    ).reshape(
        -1,           # as many sites as are available
        365,          # days
        len(metrics)) # metrics

m[0, 0:5, :]

In [ ]:
# let's also allow for an operations string so that we can _request_ data in a specific format instead of changing it after the fact.
ops_string = 's d m -> s m d'

ops = ops_string.replace(' ', '').split('->')


dim_order = [i
 for e in ops[1]
 for i in range(len(ops[0]))
 if e == ops[0][i]
 ]


m.shape, einops.rearrange(m, ops_string).shape

In [ ]:
ref = {
    'lookup_names': metadata,
    'data_names': metrics,
    'data_dims': [['site', 'days', 'metrics'][i] for i in dim_order] # use the ops string to figure out the order that will be returned
}
# to help avoid confustion later, we'll return some reference information too
# (lookup, m, ref)

In [ ]:
# self.ssurgo_data = self.ssurgo_data.reset_index(drop = True)
s = x.ssurgo_data

metadata = [
    'SoilIdx',
    ]
metrics = [
    'BD',
    'AirDry',
    'LL15',
    'DUL',
    'SAT',
    'KS',
    'Carbon',
    'SoilCNRatio',
    'FOM',
    'FOM.CN',
    'FBiom',
    'FInert',
    'NO3N',
    'NH4N',
    'PH',
    'ParticleSizeClay',
    'ParticleSizeSilt',
    'ParticleSizeSand',
    'Maize.KL',
    'Maize.LL',
    'Maize.XF',
    # 'Soybean.KL',
    # 'Soybean.LL',
    # 'Soybean.XF',
    # 'Wheat.KL',
    # 'Wheat.LL',
    # 'Wheat.XF'
    ]

s.head()

In [ ]:
# check that the soil slices are equally sized
assert len(s.Thickness.unique()) == 1
print(f'Soil compartment thickness: {s.Thickness[0]}')

In [ ]:
# check that there are an equal number of slices
_  = s.loc[:, ['SoilIdx', 'Thickness']].groupby(['SoilIdx']).count().reset_index()
assert _.Thickness.min() == _.Thickness.max()

In [ ]:
# turn range into start of slice
s['Depth'] = s['Depth'].str.split(pat = '-', expand = True)[0].astype(int)

In [ ]:
s = s.sort_values(['SoilIdx', 'Depth']).reset_index(drop = True)

lookup = s.loc[:, metadata].drop_duplicates().to_numpy()

s = s.loc[:, metrics].to_numpy()
s = s.reshape(-1, lookup.shape[0], len(metrics))

In [ ]:
ref = {
    'lookup_names': metadata,
    'data_names': metrics,
    'data_dims': ('site', 'depth', 'metrics')
}
# to help avoid confustion later, we'll return some reference information too
# (lookup, s, ref)



In [ ]:
# self.Genotypes   = self.Genotypes.reset_index(  drop = True)
g = x.Genotypes

# Genotypes are easy to deal with because the table is cleaned up in filtering results. 

g.head()


In [ ]:

metadata = [
    'File',
    'Genotype',
    ]
metrics = [
    'Grain.MaximumGrainsPerCob.FixedValue',
    'Grain.MaximumPotentialGrainSize.FixedValue',
    'Phenology.FlagLeafToFlowering.Target.FixedValue',
    'Phenology.FloweringToGrainFilling.Target.FixedValue',
    'Phenology.GrainFilling.Target.FixedValue',
    'Phenology.Juvenile.Target.FixedValue',
    'Phenology.Maturing.Target.FixedValue',
    'Phenology.MaturityToHarvestRipe.Target.FixedValue',
    'Phenology.Photosensitive.Target.XYPairs.X__1',
    'Phenology.Photosensitive.Target.XYPairs.X__2',
    'Phenology.Photosensitive.Target.XYPairs.X__3',
    'Phenology.Photosensitive.Target.XYPairs.Y__1',
    'Phenology.Photosensitive.Target.XYPairs.Y__2',
    'Phenology.Photosensitive.Target.XYPairs.Y__3',
    'Rachis.DMDemands.Structural.DMDemandFunction.MaximumOrganWt.FixedValue',
]

In [ ]:
ref = {
    'lookup_names': metadata,
    'data_names': metrics,
    'data_dims': ('genotype', 'metrics')
}
# (g.loc[:, metadata].to_numpy(), g.loc[:, metrics].to_numpy(), ref)

In [ ]:
## Example of Internals:


### Masking genotypes

The goal here is to not reinvent the wheel but to make certain tasks easier. We're storing several tables as pandas dataframes we can lean on pandas to filter the data. Filters should be applied to multiple tables so we'll automate that part to avoid forgetting to do it. The work flow will be 
1. Access a dataframe and generate a mask
2. Use a helper function to apply the mask
    1. The helper function will filter the specified dataframe (Ids or Genotypes)
    1. The helper will then use the filtered dataframe to filter the other one keeping them both in sync. 
    


In [ ]:
# Now let's select a region of the country. For this demonstration I'll filter to a region around Columbia MO.

mask = ((x.Ids.Longitude < -90) & 
        (x.Ids.Longitude > -95) &
        (x.Ids.Latitude  <  40) & 
        (x.Ids.Latitude  >  30) 
        )

print(f'Before Mask: Genotypes: {x.Genotypes.shape} Ids: {x.Ids.shape}')
x.apply_mask(table='Ids', mask=mask)
print(f'After Mask: Genotypes: {x.Genotypes.shape} Ids: {x.Ids.shape}')

In [ ]:
x.Genotypes

### Loading Results tables

The simulation results are spread across ~134 parquet files. This reduces the data storage needs but makes it harder to work with the files (than having one file per simulation or a fully fledged database server). To make the process of loading data faster, we find all the unique parquet files to be read (we might want multiple simulations from one file) and then read them in turn. Furthermore, we might only want to use a subset of years so we'll allow for this to be specified.  

In [ ]:
x.load_results(years = [1990, 2000, 2010, 2020], dry_run = True) # years is optional. If an empty list is passed all years will be returned.

In [ ]:
x.load_results(years = [1990, 2000, 2010, 2020], dry_run = False)

In [ ]:
# now results can be accessed 
x.results.shape

In [ ]:
x.results.head()

In [ ]:
### Remove run-on simulation results and return each year's results

In [ ]:
# Turn results into tensor
# Convert unix date to year/doy
tmp = x.results.merge(_prep_unix_epoch_to_date(max_year = 2025).loc[:, ['Unix', 'Year', 'DOY']].rename(columns={'Unix':'Date'}))

tmp.head()


### Masking genotypes

The goal here is to not reinvent the wheel but to make certain tasks easier. We're storing several tables as pandas dataframes we can lean on pandas to filter the data. Filters should be applied to multiple tables so we'll automate that part to avoid forgetting to do it. The work flow will be 
1. Access a dataframe and generate a mask
2. Use a helper function to apply the mask
    1. The helper function will filter the specified dataframe (Ids or Genotypes)
    1. The helper will then use the filtered dataframe to filter the other one keeping them both in sync. 
    


In [ ]:
# Now let's select a region of the country. For this demonstration I'll filter to a region around Columbia MO.

mask = ((x.Ids.Longitude < -90) & 
        (x.Ids.Longitude > -95) &
        (x.Ids.Latitude  <  40) & 
        (x.Ids.Latitude  >  30) 
        )

print(f'Before Mask: Genotypes: {x.Genotypes.shape} Ids: {x.Ids.shape}')
x.apply_mask(table='Ids', mask=mask)
print(f'After Mask: Genotypes: {x.Genotypes.shape} Ids: {x.Ids.shape}')

In [ ]:
x.Genotypes

### Loading Results tables

The simulation results are spread across ~134 parquet files. This reduces the data storage needs but makes it harder to work with the files (than having one file per simulation or a fully fledged database server). To make the process of loading data faster, we find all the unique parquet files to be read (we might want multiple simulations from one file) and then read them in turn. Furthermore, we might only want to use a subset of years so we'll allow for this to be specified.  

In [ ]:
x.load_results(years = [1990, 2000, 2010, 2020], dry_run = True) # years is optional. If an empty list is passed all years will be returned.

In [ ]:
x.load_results(years = [1990, 2000, 2010, 2020], dry_run = False)

In [ ]:
# now results can be accessed 
x.results.shape

In [ ]:
x.results.head()

### Remove run-on simulation results and return each year's results

In [ ]:
# Turn results into tensor
# Convert unix date to year/doy
tmp = x.results.merge(_prep_unix_epoch_to_date(max_year = 2025).loc[:, ['Unix', 'Year', 'DOY']].rename(columns={'Unix':'Date'}))

tmp.head()

In [ ]:
# make sure we don't allow for observations to cross over two years
# tmp2 = tmp.loc[:, ['FactorialUID', 'Year']].drop_duplicates()

tmp.loc[tmp.FactorialUID == 14689, ]

In [ ]:
import matplotlib.pyplot as plt
_ = tmp.loc[tmp.FactorialUID == 14689, ]
plt.scatter(x = _.DOY, y = _['Maize.AboveGround.Wt'])

# some of these run into the subsequent year. To deal with this...

In [ ]:
# find the smallest above ground weight within each year, usethat to get the first date within eacy year and then filter for the values that are >= the within year start.
_ = tmp.loc[:, ['FactorialUID', 'Year', 'Maize.AboveGround.Wt']].groupby(['FactorialUID', 'Year']).agg('min').reset_index()
_ = _.merge(tmp).loc[:, ['FactorialUID', 'Year', 'Date']].rename(columns={'Date': 'YearStart'})
tmp = tmp.merge(_, how='left')
tmp.head()

In [ ]:
# Retained values
_ = tmp.loc[((tmp.FactorialUID == 14689) & 
             (tmp.Date >= tmp.YearStart)
             ), ]
plt.scatter(x = _.DOY, y = _['Maize.AboveGround.Wt'])

_ = tmp.loc[((tmp.FactorialUID == 14689) & 
             (tmp.Date < tmp.YearStart)
             ), ]
plt.scatter(x = _.DOY, y = _['Maize.AboveGround.Wt'])

In [ ]:
# Turn results into tensor
tmp = x.results.merge(_prep_unix_epoch_to_date(max_year = 2025).loc[:, ['Unix', 'Year', 'DOY']].rename(columns={'Unix':'Date'}))

# find the smallest above ground weight within each year, usethat to get the first date within eacy year and then filter for the values that are >= the within year start.
_ = tmp.loc[:, ['FactorialUID', 'Year', 'Maize.AboveGround.Wt']].groupby(['FactorialUID', 'Year']).agg('min').reset_index()
_ = _.merge(tmp).loc[:, ['FactorialUID', 'Year', 'Date']].rename(columns={'Date': 'YearStart'})
tmp = tmp.merge(_, how='left')

tmp = tmp.loc[(tmp.Date >= tmp.YearStart), ]

tmp=tmp.drop(columns=['YearStart', 'Date'])

In [ ]:
tmp

In [ ]:
# if making point estimates:
tmp.drop(columns='DOY').groupby(['FactorialUID', 'Year']).agg('max').reset_index()

In [ ]:
tmp

In [ ]:
mask = (
    (tmp.FactorialUID == 11811) & 
    (tmp.Year == 1990))

min_doy, max_doy = tmp.loc[mask, 'DOY'].agg(('min', 'max')).values

out = torch.zeros(365, 3)
out[min_doy:(max_doy+1), ] = torch.tensor(tmp.loc[mask, ['Maize.AboveGround.Wt', 'Maize.LAI', 'yield_Kgha']].to_numpy())

# propagate forward observations to all dates following the max
out[max_doy:365, ] = out[max_doy, ]

plt.scatter(x = np.linspace(0, 365, 365), y = out[:, 2].numpy())

In [ ]:
x.results_as_arrays(output_type = 'point')

In [ ]:
x.results_as_arrays(output_type = 'sequence')

In [ ]:
#TODO write a method to return the 
# - cultivar information 
# - planting date as DOY
# - lookup(s)

#TODO work with the environmental data
# - reduce columns
# - create easy lookup 


## Clean up Environmental Data

In [ ]:
env_path = '/home/Shared/apsimx_env_data'+'/'

# ./apsimx_i_soil_ssurgo_profile.parquet

# met files (easy to parse)
# env_path+'apsimx_i_soil_POWER_met.parquet'

lookup = pq.read_table(env_path+'apsimx_i_soil_gps.parquet').to_pandas()

In [ ]:
# load env data
met_data = pq.read_table(env_path+'apsimx_i_soil_POWER_met.parquet').to_pandas().rename(columns = {'latitude':'Latitude', 
                                                                                                   'longitude': 'Longitude'})
met_data

ssurgo_data = pq.read_table(env_path+'apsimx_i_soil_ssurgo_profile.parquet').to_pandas().rename(columns = {'soil_i':'SoilIdx'})
ssurgo_data

In [ ]:
# shared soilIdx

Ids_idxs = x.Ids.loc[:, ['Longitude', 'Latitude', 'SoilIdx']].drop_duplicates()
ssurgo_s = ssurgo_data.loc[:, ['SoilIdx']].drop_duplicates()
met_lonlat = met_data.loc[:, ['Longitude', 'Latitude']].drop_duplicates()

In [ ]:
print(f'Uniq. Lon/Lat/Soil: {Ids_idxs.shape[0]}')
print('Condition on data availability:')
Ids_idxs = Ids_idxs.merge(ssurgo_s, how = 'inner').merge(met_lonlat, how = 'inner')
print(f'Uniq. Lon/Lat/Soil: {Ids_idxs.shape[0]}')

In [ ]:
# reduce the environmental datasets:
met_data    = Ids_idxs.drop(columns=['SoilIdx']).drop_duplicates().merge(met_lonlat, how = 'inner')
ssurgo_data = Ids_idxs.drop(columns=['Longitude', 'Latitude']).drop_duplicates().merge(ssurgo_data, how = 'inner')

In [ ]:
# update Ids & results:
x.Ids = Ids_idxs.merge(x.Ids)

_ = x.Ids.loc[:, ['FactorialUID']].drop_duplicates()
x.results = _.merge(x.results, how='inner')

In [ ]:
Ids_s    = x.Ids.loc[:, ['SoilIdx']].drop_duplicates()
x.Ids


In [ ]:
Ids

In [ ]:
# using inner join here to make sure there aren't any indexes in the results that we don't have environmental data for
ssurgo_data.merge(Ids_s, how='inner')

In [ ]:
Ids_s.merge(ssurgo_data, how='inner')

### Physical Properties


In [ ]:
env_tables = [
    'apsimx_i_soil_Soils_CERESSoilTemperature.parquet',
    'apsimx_i_soil_Soils_Chemical.parquet',
    'apsimx_i_soil_Soils_Nutrients.Nutrient.parquet',
    'apsimx_i_soil_Soils_Organic.parquet',
    'apsimx_i_soil_Soils_Physical.parquet',
    'apsimx_i_soil_Soils_Soil.parquet',
    'apsimx_i_soil_Soils_Solute.parquet',
    'apsimx_i_soil_Soils_Water.parquet',
    'apsimx_i_soil_WaterModel_WaterBalance.parquet']

soil_dict = {e:pq.read_table(env_path+e).to_pandas() for e in env_tables}#[env_tables[1]]

# focus on only one entry
soil_dict = {k:soil_dict[k].loc[(soil_dict[k].list_idx == 1), ] for k in env_tables}

In [ ]:
sorted(list(set(sum([list(pq.read_table(env_path+e).to_pandas()) for e in env_tables], []))))

# problem: There are some columns that just don't look like they are saved in the data. E.g. soil/Physical.`maize LL` 

['$type',
 'AirDry',
 'AirDryMetadata',
 'ApsoilNumber',
 'BD',
 'BDMetadata',
 'CN2Bare',
 'CNCov',
 'CNRed',
 'Carbon',
 'CarbonUnits',
 'CatchmentArea',
 'Comments',
 'Country',
 'D0',
 'DUL',
 'DULMetadata',
 'DataSource',
 'DepthConstant',
 'DiffusConst',
 'DiffusSlope',
 'DischargeWidth',
 'Enabled',
 'Exco',
 'FBiom',
 'FIP',
 'FInert',
 'FOM',
 'FOMCNRatio',
 'FilledFromTop',
 'InitialPAWmm',
 'InitialValues',
 'InitialValuesUnits',
 'KS',
 'LL15',
 'LL15Metadata',
 'Latitude',
 'Longitude',
 'MaxDepthSoluteAccessible',
 'MaxEffectiveRunoff',
 'Name',
 'PH',
 'PHMetadata',
 'PHUnits',
 'PSIDul',
 'ParticleSizeClay',
 'ParticleSizeSand',
 'ParticleSizeSilt',
 'ReadOnly',
 'RecordNumber',
 'Region',
 'RelativeTo',
 'ResourceName',
 'RunoffEffectivenessAtMovingSolute',
 'SAT',
 'SATMetadata',
 'SWCON',
 'Salb',
 'SoilCNRatio',
 'SoilType',
 'State',
 'SummerCona',
 'SummerDate',
 'SummerU',
 'Thickness',
 'WaterTableConcentration',
 'WinterCona',
 'WinterDate',
 'WinterU',
 'YearOfSampling',
 'list_idx']

In [ ]:
soil_dict['apsimx_i_soil_Soils_Physical.parquet']

In [ ]:
# table to cols:
desired_fields = {
# 'apsimx_i_soil_Soils_CERESSoilTemperature.parquet': [],
'apsimx_i_soil_Soils_Chemical.parquet': ['Thickness', 'PH'],
# 'apsimx_i_soil_Soils_Nutrients.Nutrient.parquet': [],
'apsimx_i_soil_Soils_Organic.parquet': ['Thickness', 'Carbon', 'SoilCNRatio', 'FBiom', 'FInert', 'FOM', 'FOMCNRatio', 'CarbonUnits'],
'apsimx_i_soil_Soils_Physical.parquet': ['Thickness', 'ParticleSizeClay', 'ParticleSizeSand', 'ParticleSizeSilt', 'BD', 'AirDry', 'LL15', 'DUL', 'SAT'],
# 'apsimx_i_soil_Soils_Soil.parquet': [],
# 'apsimx_i_soil_Soils_Solute.parquet': [],
'apsimx_i_soil_Soils_Water.parquet': ['Thickness', 'InitialValues', 'InitialPAWmm', 'RelativeTo'],
'apsimx_i_soil_WaterModel_WaterBalance.parquet': ['Thickness', 'SWCON']
}

In [ ]:
res = [soil_dict[k].loc[:, desired_fields[k]] for i, k in enumerate(desired_fields)]


# pd.merge( res[1])
res[0]
res[1]

In [ ]:
soil_dict[env_tables[1]] 	
soil_dict[env_tables[3]] 	

# soil_dict['apsimx_i_soil_Soils_CERESSoilTemperature.parquet']
soil_dict['apsimx_i_soil_Soils_Chemical.parquet'] # Thickness	PH 
# soil_dict['apsimx_i_soil_Soils_Nutrients.Nutrient.parquet']
soil_dict['apsimx_i_soil_Soils_Organic.parquet'] #Thickness	Carbon	SoilCNRatio	FBiom	FInert	FOM	$type	FOMCNRatio	CarbonUnits
soil_dict['apsimx_i_soil_Soils_Physical.parquet'] # 	Thickness	ParticleSizeClay	ParticleSizeSand	ParticleSizeSilt	BD	AirDry	LL15	DUL	SAT	
# soil_dict['apsimx_i_soil_Soils_Soil.parquet'] # contains metadata
# soil_dict['apsimx_i_soil_Soils_Solute.parquet'] # # Maybe useful, but it dosn't look like it? Thickness	InitialValues	Exco	FIP		InitialValuesUnits	WaterTableConcentration	D0	DepthConstant	MaxDepthSoluteAccessible	RunoffEffectivenessAtMovingSolute	MaxEffectiveRunoff	Name
soil_dict['apsimx_i_soil_Soils_Water.parquet'] # Thickness	InitialValues	$type	InitialPAWmm	RelativeTo
soil_dict['apsimx_i_soil_WaterModel_WaterBalance.parquet'] # 	Thickness	SWCON	

In [ ]:
# Depth	Thickness	BD	AirDry	LL15	DUL	SAT	KS	Carbon	SoilCNRatio	FOM	FOM.CN	FBiom	FInert	NO3N	NH4N	PH	ParticleSizeClay	ParticleSizeSilt	ParticleSizeSand	Maize.KL	Maize.LL	Maize.XF	Soybean.KL	Soybean.LL	Soybean.XF	Wheat.KL	Wheat.LL	Wheat.XF


## Define working set

In [ ]:
# Starting with a location and year

import plotly.express as px

fig = px.scatter_mapbox(Ids.loc[:, ['Longitude', 'Latitude']].drop_duplicates(), lon = 'Longitude', lat = 'Latitude',
                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# -> File	Genotype  --Genotypes-> 
#	 FactorialUID     --Results->

lon, lat, soil = [-76.65611874999999, 42.733264, 141] #lon lat soil

# should allow ranges, slices, or all
sow = '19-Jun'
# allow all
cultivar = 'Cultivar1'


# -> File FactorialUID

mask = (
    (Ids.Longitude == lon) &
    (Ids.Latitude == lat) &
    (Ids.SoilIdx == soil) &
    (Ids.SowDate == sow) &
    (Ids.Genotype == cultivar))

Ids.loc[mask, ]

In [ ]:
# Starting with a target set of genotypes
mask = Genotypes['Grain.MaximumGrainsPerCob.FixedValue'] == Genotypes['Grain.MaximumGrainsPerCob.FixedValue'].max()

Genotypes.loc[mask, ['File', 'Genotype']]

## Convert Small Datasets to Tensors
### `Genotypes` (Cultivar variables)

Warning! There are some NAs from Genotypes that are not "Cultivar\d+" Genotypes. These are calibrated genotypes that with defaults that are not clear.

In [ ]:
# keep as df or matrix (contains text)
Genotypes_lookup = Genotypes.loc[:, ['File', 'Genotype']].copy()

# ['Grain.MaximumGrainsPerCob.FixedValue',
#  'Grain.MaximumPotentialGrainSize.FixedValue',
#  'Phenology.FlagLeafToFlowering.Target.FixedValue',
#  'Phenology.FloweringToGrainFilling.Target.FixedValue',
#  'Phenology.GrainFilling.Target.FixedValue',
#  'Phenology.Juvenile.Target.FixedValue',
#  'Phenology.Maturing.Target.FixedValue',
#  'Phenology.MaturityToHarvestRipe.Target.FixedValue',
#  'Phenology.Photosensitive.Target.XYPairs.X__1',
#  'Phenology.Photosensitive.Target.XYPairs.X__2',
#  'Phenology.Photosensitive.Target.XYPairs.X__3',
#  'Phenology.Photosensitive.Target.XYPairs.Y__1',
#  'Phenology.Photosensitive.Target.XYPairs.Y__2',
#  'Phenology.Photosensitive.Target.XYPairs.Y__3',
#  'Rachis.DMDemands.Structural.DMDemandFunction.MaximumOrganWt.FixedValue']

Genotypes_cols = list(Genotypes)
Genotypes = Genotypes.drop(columns=['File', 'Genotype'])
# coerce None to NaN so we can convert to matrix
for e in list(Genotypes):
    Genotypes[e] = Genotypes[e].astype(float)

# Genotypes = torch.tensor(Genotypes.to_numpy())

In [ ]:
Genotypes_lookup

In [ ]:
# for a given idx and year...
idx_Ids = 1
year = 2000

In [ ]:
# get lookup information
lookup = Ids.loc[idx_Ids, ].to_dict()

In [ ]:
mask = ((Genotypes_lookup.File == lookup['File']) &  (Genotypes_lookup.Genotype == lookup['Genotype']))
# should only have a single value
assert sum(mask) == 1

idx_Geno = Genotypes_lookup.loc[mask, ].index[0]

Genotypes[idx_Geno]

In [ ]:
lookup

In [ ]:
lookup_date = _prep_unix_epoch_to_date(max_year = 2024)
lookup_date.head()

In [ ]:
Result = pq.read_table(apsimx_sim_parquet_dir+'/'+'sim_1698440407_4739.parquet').to_pandas()

In [ ]:
# lookup_* is a internally generated ref
# *_lookup is a table based on loaded apsimx data

In [ ]:
Result_lookup = Result.loc[:, ['Date', 'FactorialUID']].copy()
Result.drop(columns=['Date', 'FactorialUID'])

Result_list = list(Result)
Result = torch.tensor(Result.to_numpy())

Result_lookup.head()

In [ ]:
print(Result_lookup.shape)
Result_lookup.merge(lookup_date.rename(columns={'Unix':'Date'}), how = 'left')

In [ ]:
Result_lookup.loc[(Result_lookup.FactorialUID == 24024), ].merge(lookup_date.rename(columns={'Unix':'Date'}), how = 'left').Date.max()

In [ ]:
((19250-5285)/365)+1984

In [ ]:
Genotypes_cols

In [ ]:
lookup

In [ ]:
# TODO make this valid for torch
sow_date = lookup_date.loc[((lookup_date.Year == year) & 
                            (lookup_date.SowDate == lookup['SowDate'])), 'Unix'].values[0]

# because of how this is set up index is also valid
_ = lookup_date.loc[(lookup_date.Year == year), 'Unix'].agg(['min', 'max'])
_['min']

In [ ]:
mask = (
    (Result_lookup['Date'] >= _['min']) & 
    (Result_lookup['Date'] <= _['max']) &
    (Result_lookup['FactorialUID'] == lookup['FactorialUID'])
    )

idx_Result = Result_lookup.loc[mask, ].index

Result[idx_Result, ].shape

In [ ]:
#TODO make sure there aren't any values before the SowDate
Result_list

In [ ]:
start = 129486



px.imshow((Result[(start-30):(start+30), 1:-1].numpy()[0:30, ]).transpose())

In [ ]:
import matplotlib.pyplot as plt

px.imshow(Result[idx_Result, ].numpy()[0:30, ])

In [ ]:
out = torch.zeros((365, 6))

Result_lookup.loc[mask, ['Date']].min().values[0] - _['min']

# sow_date - _['min']

In [ ]:
datetime.datetime(1970, 1, 1, 0, 0) + datetime.timedelta(10957)

In [ ]:


# idx_Result.to_list()
# Result

In [ ]:
Genotypes_cols